## Automating Disaster Mapping Processes
### Cases from flood disasters in the cities of the Global South

This code will turn raster data into an interactive web map dispalying the extent of flooding events and its effects on the local population and infrastructure.

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
import osmnx as ox
import shapely
from shapely.geometry import Point, LineString, Polygon, shape
from shapely.geometry import box
import rasterio as rio
import rasterio.features
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.features import shapes
from rasterio.mask import mask
from fiona.crs import from_epsg
import pycrs
import os
import mapclassify

### Raster transformations

In [64]:
# Filepath for WSF raster file
wsf_fp = r'/Users/ohtonygren/Yliopisto/Gradu/Data/WSF/Bangkok_WSF2019population_10m.tif'

wsf_raster = rio.open(wsf_fp)

# Filepath for ICEYE raster file
iceye_fp = r'/Users/ohtonygren/Yliopisto/Gradu/Data/ICEYE/Bangkok_iceye.tif'

iceye_orig = rio.open(iceye_fp)

In [66]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Getting CRS from the WSF data
dstCrs = wsf_raster.crs

# Calculate transform array and shape of reprojected layer
transform, width, height = calculate_default_transform(iceye_orig.crs, dstCrs, iceye_orig.width, iceye_orig.height, *iceye_orig.bounds)

# Working of the meta for the destination raster
kwargs = iceye_orig.meta.copy()
kwargs.update({'crs': dstCrs, 'transform': transform, 'width': width, 'height': height})

iceye_raster = rio.open(iceye_fp, 'w', **kwargs)

# Reproject and save raster band data
for i in range(1, iceye_raster.count +1):
    reproject(
        source=rio.band(iceye_orig, i),
        destination=rio.band(iceye_raster, i),
        src_crs=iceye_raster.crs,
        dstCrs=dstCrs,
        resampling=Resampling.nearest)

# Close destination raster    
iceye_raster.close()

print('Progress: Raster reprojection done.')

# Reopening the raster that is now projected to EPSG: 4326
iceye_wgs84 = rio.open(iceye_fp)

Progress: Raster reprojection done.


### Vector transformations

In [67]:
# Polygonizing the raster file
mask = None
with rio.Env():
    with rio.open(iceye_fp) as src:
        image = src.read(1) # first band
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)))
        
geoms = list(results)

# Creating a GeoDataFrame from the polygonized raster
iceye_breaks = gpd.GeoDataFrame.from_features(geoms)

# Get indexes where 'raster_val' column has value 'min'
indexNames = iceye_breaks[iceye_breaks['raster_val'] == iceye_breaks['raster_val'].min()].index
# Delete 'min' rows indexes from dataframe as they are Null values in the raster file
iceye_breaks.drop(indexNames, inplace=True)

print('Progress: Polygonizing done.')

Progress: Polygonizing done.


In [70]:
# Rounding up unnecessary decimals
iceye_breaks['raster_val'].round(3)

# Removing all rows with the value of zero
iceye_breaks[iceye_breaks.raster_val != 0.000]

# Classifying the data by natural breaks into 6 classes
breaks = mapclassify.NaturalBreaks.make(k=6)
iceye_breaks['naturalBreaks'] = iceye_breaks[['raster_val']].apply(breaks)

# Dissolving the data into the 6 classified breaks
iceye_breaks.dissolve(by='naturalBreaks', as_index=False)

,naturalBreaks,geometry,raster_val
0,0,"MULTIPOLYGON (((100.33636 13.67740, 100.33636 ...",0.128244
1,1,"MULTIPOLYGON (((100.33911 13.67273, 100.33911 ...",1.083285
2,2,"MULTIPOLYGON (((100.33993 13.65513, 100.33966 ...",1.762310
3,3,"MULTIPOLYGON (((100.34571 13.64715, 100.34571 ...",1.938872
4,4,"MULTIPOLYGON (((100.44416 13.53907, 100.44416 ...",6.357238
5,5,"MULTIPOLYGON (((100.40759 13.71288, 100.40759 ...",9.343607


### Raster clipping and affected population estimation

In [8]:
results = []

for i in iceye_breaks['naturalBreaks']:
        
    roi = iceye_breaks[iceye_breaks.naturalBreaks == i]
        
    gtraster, bound = rio.mask.mask(wsf_raster, roi['geometry'], crop=True)
        
    results.append(gtraster[0][gtraster[0]>0].sum())
    
iceye_breaks['population'] = results

# Dividing by 1000 because the WSF data has been multiplied by 1000 to save the file as integer
iceye_breaks['population'] = iceye_breaks['population'].div(1000).round(0)

In [9]:
iceye_breaks

,naturalBreaks,geometry,raster_val,population
0,0,"MULTIPOLYGON (((100.33636 13.67740, 100.33636 ...",0.128,853400.0
1,1,"MULTIPOLYGON (((100.33911 13.67273, 100.33911 ...",1.083,381778.0
2,2,"MULTIPOLYGON (((100.33966 13.65568, 100.33966 ...",1.762,152200.0
3,3,"MULTIPOLYGON (((100.34571 13.64715, 100.34571 ...",1.939,47705.0
4,4,"MULTIPOLYGON (((100.44416 13.53907, 100.44416 ...",6.357,4238.0
5,5,"MULTIPOLYGON (((100.40759 13.71288, 100.40759 ...",9.344,824.0


#### Getting OSM data from flooded area

In [37]:
# Creating a constant value to use for dissolve
iceye_breaks['dissolve'] = 1

# Dissolving all the flooded polygons into one for extent of flooding
iceye_dissolve = iceye_breaks.dissolve(by='dissolve')

# Turning the GeoDataFrame into a shapely polygon for OSMnx
osm_aoi = flood_aoi.iloc[0]['geometry']

# IF THERE ARE ISSUES WITH THE SCRIPT, TRY REMOVING THE FOLLOWING PIECE OF CODE
# Ignore Shapely deprecation warnings to clean up code, warning are due to the OSMnx module and future Shapely 2.0 upgrade 
import warnings
warnings.filterwarnings("ignore")

# Getting OpenStreetMap data from the flooded area
hospital = ox.geometries.geometries_from_polygon(osm_aoi, tags={'amenity':'hospital'})
pharmacy = ox.geometries.geometries_from_polygon(osm_aoi, tags={'amenity':'pharmacy'})
buildings = ox.geometries.geometries_from_polygon(osm_aoi, tags={'building': True})

# Removing unnecessary columns
buildings = buildings[['geometry', 'building', 'name:en']]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/osmnx/utils_geo.py:335: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part ge

In [14]:
buildings = buildings[['geometry', 'building']]

In [15]:
buildings = build_test[build_test.area != 0.000]

/var/folders/kt/ttp65kq11zdgh8vfjd7dtgm40000gn/T/ipykernel_1722/2227512806.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  build_test = build_test[build_test.area != 0.000]


## FOLIUM

In [48]:
# Reprojecting the data to a CRS that uses a metric system instead of degrees for the buffering
hospital = hospital.to_crs(epsg=3857)
pharmacy = pharmacy.to_crs(epsg=3857)

# Calculating the centroids of all the points & polygons in the pharmacy dataset
hospital['geometry'] = hospital['geometry'].centroid
pharmacy['geometry'] = pharmacy['geometry'].centroid

# Reprojecting the data back to a CRS that is better for plotting
hospital = hospital.to_crs(epsg=4326)
pharmacy = pharmacy.to_crs(epsg=4326)

# Getting x and y coordinates of hospitals for mapping
hospital['x'] = hospital['geometry'].apply(lambda geom: geom.x)
hospital['y'] = hospital['geometry'].apply(lambda geom: geom.y)

# Getting x and y coordinates of pharmacies for mapping
pharmacy['x'] = pharmacy['geometry'].apply(lambda geom: geom.x)
pharmacy['y'] = pharmacy['geometry'].apply(lambda geom: geom.y)

# Creating a list of coordinate pairs
hospitals = list(zip(hospital['y'], hospital['x']))
pharmacies = list(zip(pharmacy['y'], pharmacy['x']))

# Adding clusters of points to map
hospital_cluster = MarkerCluster()
pharma_cluster = MarkerCluster()

# Adding custom labels and icons to every hospital and pharmacy location
for point in range(0, len(hospitals)):
    folium.Marker(hospitals[point], popup=hospital.iloc[point]['name:en'], icon=folium.Icon(color='red', icon='fa-h-square', prefix='fa')).add_to(hospital_cluster)
    
for point in range(0, len(pharmacies)):
    folium.Marker(pharmacies[point], popup=pharmacy.iloc[point]['name:en'], icon=folium.Icon(color='green', icon='fa-medkit', prefix='fa')).add_to(pharma_cluster)

In [57]:
# Creating Folium map
m = folium.Map(location=[13.7, 100.6],
              zoom_start=11,
              control_scale=True,
              tiles='CartoDB Positron')

# Adding clusters to map
hospital_cluster.add_to(m)
pharma_cluster.add_to(m)

# Setting a CRS to the data
iceye_breaks.crs = 'epsg:4326'

# Creating a Geo-id that Folium needs for plotting, it needs to have a unique indetifier for each row (Tenkanen & al. 2022)
iceye_breaks['geoid'] = iceye_breaks.index.astype(str)

# Plotting flood data
folium.Choropleth(geo_data=iceye_breaks,
                 name='Flood map',
                 data=iceye_breaks,
                 columns=['geoid', 'naturalBreaks'],
                 key_on='feature.id',
                 fill_color='Blues',
                 fill_opacity=0.7,
                 line_opacity=0.2,
                 line_color='white',
                 line_weight=0,
                 highlight=False,
                 smooth_factor=1.0,
                 legend_name='Flood in Bangkok').add_to(m)

# Adding building footprints to map
build_j = buildings.to_json()
build_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'fillColor': 'red', 'color': 'red'})
build_j.add_to(m)

In [63]:
m.save(outfile = 'test.html')

### MAPBOX

In [ ]:
test_gjson = test.to_json()

In [ ]:
token = os.getenv('pk.eyJ1Ijoib2h0b255Z3JlbiIsImEiOiJja2tyMmowZ2YwZTU4MndvNm4yMW84OXhrIn0.lwnlCLxsvFIPqn7yzyGxXw')

In [ ]:
from mapboxgl.viz import *
from mapboxgl.utils import *
import mapclassify

In [ ]:
color_stops = create_color_stops([0,1,2,3,4,5,6], colors='YlGnBu')

viz = ChoroplethViz('iceye_breaks', 
                    access_token=token, 
                    color_property = 'naturalBreaks', 
                    color_stops = color_stops, 
                    center = (13.7, 100.6), 
                    zoom=11)

### LEAFMAP

In [ ]:
import os
import leafmap.leafmap as leafmap

In [ ]:
#out_dir = os.path.expanduser('~/Yliopisto/Gradu/Data/ICEYE/')

#if not os.path.exists(out_dir):
    #os.makedirs(out_dir)

#flood_raster = os.path.join(out_dir, 'Bangkok_iceye.tif')

In [ ]:
out_dir = os.path.expanduser('~/Yliopisto/Gradu/Data/ICEYE/')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

flood_raster = os.path.join(out_dir, 'Bangkok_iceye.tif')

In [ ]:
Map = leafmap.Map()

In [ ]:
Map.add_raster(flood_raster, colormap='coolwarm', layer_name='Flood')

In [ ]:
Map.to_html('Thesis_map.html')

### KEPLER

In [ ]:
from keplergl import KeplerGl
map_1 = KeplerGl()